### Exercise 4
## Strongly and weakly relevant features
# G. Bontempi


## Question

Let us consider a regression task with 
$N=200$ samples, $n=50$ input features (in the matrix $\tt{X}$) and one target variable (vector $\tt{Y}$).

Knowing that there are 3 strongly relevant variables and 2 weakly relevant variables,
the student has to define and implement a strategy to find them.

No existing feature selection code has to be used. However, 
the student may use libraries to implement supervised learning algorithms.

The student code should 

* return the position of the 3 strongly relevant variables and 2 weakly relevant variables,
* discuss what strategy could have been used if the number
of strongly and weakly variables was not known in advance.

## Data generation

Let us see first how the input-output dataset was generated.
The knowledge of the stochastic process
generating the data will allow us to define the correct set of strongly and weakly relevant features.


In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import scipy.io


# Set random seed
np.random.seed(0)

# Initialize parameters
N = 200
n = 50
strong = [0, 6, n-1]  # Converting to 0-based indexing
weak = [7, 8]  # Converting to 0-based indexing
irr = list(set(range(n)) - set(strong) - set(weak))
ns = len(strong)
nw = len(weak)

# Generate random data
Xw = np.random.normal(size=(N, nw))
X = np.random.normal(size=(N, n))

# Create strong features
X[:, strong[0]] = np.sum(np.abs(Xw), axis=1) + np.random.normal(0, 0.1, N)
X[:, strong[1]] = np.prod(np.abs(Xw), axis=1) + np.random.normal(0, 0.1, N)
X[:, strong[2]] = np.log(np.prod(np.abs(Xw), axis=1)) + np.random.normal(0, 0.1, N)

# Set weak features
X[:, weak] = Xw

# Scale the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Generate response variable Y
Y = np.sum(np.abs(X[:, strong]), axis=1) + np.random.normal(0, 0.1, N)




The relationship between ${\mathbf X}=\{{\mathbf x_1},{\mathbf x_2},\dots,{\mathbf x_{50}}\}$
and ${\mathbf y}$ is given by

\begin{equation}
\label{eq:model}
{\mathbf y}=|x_1+x_7+x_{50}|+{\mathbf w}
\end{equation}

where  ${\mathbf x_1}=| x_8+x_9|+{\mathbf w_1}$, 
${\mathbf x_7}=| x_8 x_9|+{\mathbf w_7}$, 
${\mathbf x_{50}}=\log | x_8 x_9| +{\mathbf w_{50}}$
and ${\mathbf w},{\mathbf w_1},{\mathbf w_7},{\mathbf w_{50}}$, are all Normal with zero mean and standard deviation $0.1$.

## Definition of strongly and weakly relevant features

In the course a strongly relevant feature is defined as a feature ${\mathbf x}_j$ such that
$$ I({\mathbf X}_{-j},{\mathbf y})< I({\mathbf X},{\mathbf y})$$ or equivalently

$$ H({\mathbf y}| {\mathbf X}_{-j})> H({\mathbf y}|{\mathbf X})$$
By removing a strongly relevant feature from the input set, the conditional variance of ${\mathbf y}$ increases.

From the model definition it follows that 
$$ p(y| X)=p(y| x_1,x_7,x_{50})$$
or equivalently that ${\mathbf y}$
is conditionally independent of all the other variables when the value of $\{x_1,x_7,x_{50}\}$ is known.

The set of strongly relevant variables (which is also the Markov blanket) is  then $\{{\mathbf x_1},{\mathbf x_7},{\mathbf x_{50}}\}$. 

A weakly relevant feature is a feature  ${\mathbf x_j}$ that is not strongly relevant and such that 
$$ I({\mathbf S}_{-j},{\mathbf y})< I({\mathbf S},{\mathbf y})$$ 
or equivalently 
$$ H({\mathbf y}| {\mathbf S}_{-j})> H({\mathbf y}|{\mathbf S})$$
for a certain context $S \subset X$. If we consider $S= X \setminus \{x_1,x_7,x_{50}\}$ then 
$$ p(y| S)=p(y| x_8,x_9)$$
It follows that ${\mathbf y}$
is conditionally independent of all the other features of $S$ when the value of $\{x_8,x_9\}$ is known.

The set of weakly relevant variables is  then $\{{\mathbf x_8},{\mathbf x_9}\}$. 

In other terms the set of weakly relevant variables $\{x_8,x_9\}$  provides information about ${\mathbf y}$ for some contexts, e.g. the contexts where $\{x_1,x_7,x_{50}\}$ are not available.

All the other features are irrelevant since they play no role in the dependency between ${\mathbf X}$ and ${\mathbf y}$.

## Data-driven estimation of conditional entropy

In the real setting (i.e. the one the student is confronted with) the conditional probability and the relationships between input features is not accessible.
It is not then possible to compute analytically the information or the entropy terms.

Nevertheless, it is possible to estimate the conditional probability  $p(y|S)$
and consequently the conditional entropy term $H({\mathbf y}| {\mathbf S})$ for a subset $S$ of 
features by making some assumptions:

1. we have a learning model able to return an unbiased and low variant estimation
of the regression function. In this case the estimated MISE returns  a good approximation of the conditional variance (i.e. the noise variance) 
2. the conditional probability is Gaussian. In this case there is a direct link between the conditional variance and the conditional entropy.

In other terms we make the assumption that 
$$H({\mathbf y}| {\mathbf S_1}) < H({\mathbf y}| {\mathbf S_2}) $$ if
$$\widehat{\text{MISE}_1}< \widehat{\text{MISE}_2}$$
where $\widehat{\text{MISE}_i}$ is the estimated (e.g. by leave-one-out) generalization
error of a learner trained with the input set $S_i$.


## Data-driven identification of strongly relevant features 

Here we identify in a data-driven manner the set of strongly relevant features by
choosing as learner a Random Forest and by using a holdout strategy to estimate
the generalization error. 

In practice, we 

1. remove a single input feature at the time, 
2. split the dataset in training and validation set and learn a Random Forest with the training set
3. compute the Random Forest generalization error for the validation set
4. rank the features to select the ones that induced a largest increase of the generalization error

In [3]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor



# Split indices for training and validation
Itr = np.random.choice(N, size=round(N/2), replace=False)
Ival = np.array([i for i in range(N) if i not in Itr])

# Function to mimic R's pred function with RandomForest
def pred_rf(X_train, y_train, X_test, class_output=False):
    
    rf = RandomForestRegressor(n_estimators=500)
    rf.fit(X_train, y_train)
    return rf.predict(X_test)
    

# Initial prediction and MISE computation
Yhat = pred_rf( X[Itr], Y[Itr], X[Ival])
Ehat = (Y[Ival] - Yhat) ** 2
MISEhat = ('Testing error=',np.mean(Ehat)]

print(np.mean(MISEhat ** 2))

# Initialize arrays for feature importance computation
MISEhatj = np.zeros(n)
Ehatj = np.full((len(Ival), n), np.nan)

# Compute MISE for each feature removed
for j in range(n):
    # Create views of X without column j
    X_tr_reduced = np.delete(X[Itr], j, axis=1)
    X_val_reduced = np.delete(X[Ival], j, axis=1)
    
    Yhatj = pred_rf( X_tr_reduced, Y[Itr], X_val_reduced)
    Ehatj[:, j] = (Y[Ival] - Yhatj) ** 2
    MISEhatj[j] = np.mean(Ehatj[:, j])

# Find strongly relevant features
diff_MISE = MISEhatj - MISEhat
stronghat = np.argsort(diff_MISE)[::-1][:ns]

print(f"Strongly relevant identified= {stronghat+1}")



0.0741982883175096
Strongly relevant identified= [49  0  6]


According to the procedure above, by knowing that there are _ns_ strongly relevant variables, the set of strongly relevant variables is in the columns _stronghat_ of the input matrix $X$.

## Data-driven identification of weakly relevant features 

The identification of weakly relevant variables would need a search in the space of all 
possible contexts. Here we limit to consider the context $S= X \setminus \{x_1,x_7,x_{50}\}$ obtained by removing the strongly 
relevant features from the input set. The hold-out procedure is similar to the one in the previous section.

In [6]:

# Make prediction excluding strong features
Yhat = pred_rf(X[Itr][:, [i for i in range(X.shape[1]) if i not in strong]], Y[Itr], 
            X[Ival][:, [i for i in range(X.shape[1]) if i not in strong]])

# Calculate mean squared error
wMISEhat = np.mean((Y[Ival] - Yhat) ** 2)


# Initialize array for storing MSE values
wMISEhatj = np.full(n, -100, dtype=float)

# Calculate MSE for each feature (excluding strong features)
for j in [x for x in range(n) if x not in strong]:
    # Create mask for columns to exclude (both strong features and current feature j)
    cols_to_exclude = strong + [j]
    mask = [i for i in range(X.shape[1]) if i not in cols_to_exclude]
    
    # Make prediction without current feature
    Yhatj = pred_rf( X[Itr][:, mask], Y[Itr], X[Ival][:, mask])
    wMISEhatj[j] = np.mean((Y[Ival] - Yhatj) ** 2)

# Find weakly relevant features
differences = wMISEhatj - wMISEhat
weakhat = np.argsort(differences)[::-1][:nw]  # Sort in descending order and get top nw indices
print(np.sort(differences)[::-1][:nw])  # Print the top nw differences
print(f"Weakly relevant identified= {weakhat+1}")



[0.55015245 0.0712505 ]
Weakly relevant identified= [8 9]


According to the procedure above we see that there are _nw_ features that,
once removed, increase the generalization error of the context $S= X \setminus \{x_1,x_7,x_{50}\}$. We may deduce then that the set of weakly relevant variables is in the columns _weakhat_ of the input matrix $X$.

## What to do in the general case

The solution in this exercise has been facilitated by the knowledge of
the number of strongly and weakly relevant features. Unfortunately, this information is hardly
available in real settings.

The main issue related to identification of relevant features is that
we cannot compute the analytical exact value of the conditional entropy (or conditional
information terms) because of the stochastic finite-data setting.
In practice we have only rough estimates of those terms.
Nevertheless, most of the time we are not interested in the actual values
of those terms but in their relative values: for instance we may be interested 
to know if
$$H({\mathbf y}| {\mathbf S_1}) < H({\mathbf y}| {\mathbf S_2}) $$ 
of if their difference is smaller than zero.

Since those values are only estimated the fact that 
$$\hat{H}({\mathbf y}| {\mathbf S_1}) < \hat{H}({\mathbf y}| {\mathbf S_2}) $$
does not necessarily provide enough evidence to draw a conclusion. Given the stochastic setting
a solution could be the adoption of statistical tests. For instance 
if $H$ is approximated by $\widehat{\text{MISE}}$ we could use a statistical test
to check whether the mean $\widehat{\text{MISE}_1}$ is significantly smaller than $\widehat{\text{MISE}_2}$. 

Let us see how this could be done in practice.

### Data-driven identification of the number of strongly relevant features 

In this case we do not know exactly where to stop in the decreasing ranking of the vector $\tt{MISEhatj-MISEhat}$.

In what follows we use a t-test comparing the vector of test errors 
(stored in the R variable $\tt{Ehatj}$) of 
each feature set $X_{-j}$ to the the one of $X$ (stored in the R variable $\tt{Ehat}$). This checks if
the mean $\widehat{\text{MISE}_{-j}}$ is significantly
larger (pvalue smaller than $0.01$) than $\widehat{\text{MISE}}$.

In [9]:
import numpy as np
from scipy import stats

pv = np.zeros(n)
for j in range(n):
    # Perform paired t-test and extract p-value
    t_stat, pv[j] = stats.ttest_rel(Ehatj[:,j], Ehat, alternative='greater')

# Find indices where p-value is less than 0.01
stronghat_test = np.where(pv < 0.01)[0]

print('stronghat_test=',stronghat_test+1)
# Sort p-values and get their indices
sorted_indices = np.argsort(pv)[:5]
sorted_pvalues = pv[sorted_indices]
# Print sorted p-values and their indices
for idx, p_val in zip(sorted_indices, sorted_pvalues):
    print(f"Index: {idx+1}, P-value: {p_val}")


stronghat_test= [ 1  7 50]
Index: 7, P-value: 0.0004241549364749579
Index: 50, P-value: 0.0008818805852963087
Index: 1, P-value: 0.003359186787547234
Index: 35, P-value: 0.016935582494711807
Index: 38, P-value: 0.02415069349899065


It follows that (for the given pvalue threshold) the set of strongly relevant features is _stronghat.test_. Of course this number could be different for
different pvalue thresholds.

### Data-driven identification of the number of weakly relevant features 

The procedure above can be used as well for detecting weakly relevant features
for a given context.
Nevertheless, since the number of weakly features is not given in advance, the problem of finding the set of weakly relevant features would remain much harder. 
In fact, we are not supposed to stop the search until we have not considered all the possible contexts.